## 读取数据

In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import seaborn as sns
import matplotlib
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
plt.rcParams['figure.dpi'] = 150 # 修改图片分辨率
plt.rcParams.update({'font.size': 18})
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica Neue']
import matplotlib.pyplot as plt


In [2]:
# Load the data
data = pd.read_excel('./P0022023-study2.xlsx')
data
exclusive = ['Label']
data[ exclusive] = data[ exclusive].fillna(-1)
data = data.drop(columns=['ID'])
data # display the first few rows of the processed data

,Subtype I Baseline,Subtype II Baseline,Label,Group
0,0.992366,0.007634,0,0
1,0.799419,0.200581,0,0
2,0.039474,0.960526,1,0
3,0.008023,0.991977,1,0
4,0.946649,0.053351,0,0
...,...,...,...,...
69,0.747913,0.252087,0,0
70,0.026080,0.973920,1,0
71,0.101807,0.898193,1,0
72,0.005035,0.994965,1,0


## 缺失值

In [3]:
#无缺失
data.isnull().sum()

Subtype I Baseline     0
Subtype II Baseline    0
Label                  0
Group                  0
dtype: int64

In [4]:
#均值替换
for col in data.columns:
    data[col]=data[col].fillna(data[col].mean())

In [5]:
df =data

## 数据不平衡数据

In [6]:
df[exclusive] = df[exclusive].fillna(-1)
cols=[i for i in df.columns if i not in ['Group']]
X=df[cols]
y=df['Group']

from collections import Counter
print(sorted(Counter(y).items()))
from imblearn.over_sampling import SMOTE, ADASYN

X_resampled, y_resampled = SMOTE(sampling_strategy=0.96, random_state=1).fit_resample(X, y)
# 打印处理之前的类别人数
print("处理之前的类别人数:")
print(y.value_counts())

[(0, 50), (1, 24)]
处理之前的类别人数:
0    50
1    24
Name: Group, dtype: int64


In [7]:
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0, 50), (1, 48)]


In [8]:
import pickle
# 保存这些变量到文件：X_resampled，y_resampled和cols_select
with open('data/variables-huafen.pkl', 'wb') as file:
    pickle.dump((X_resampled, y_resampled), file)


# 将数据保存为 Excel 文件
if isinstance(X_resampled, pd.DataFrame) or isinstance(X_resampled, pd.Series):
    X_resampled.to_excel('table/X_resampled.xlsx')

if isinstance(y_resampled, pd.DataFrame) or isinstance(y_resampled, pd.Series):
    y_resampled.to_excel('table/y_resampled.xlsx')

In [9]:
X_resampled

,Subtype I Baseline,Subtype II Baseline,Label
0,0.992366,0.007634,0
1,0.799419,0.200581,0
2,0.039474,0.960526,1
3,0.008023,0.991977,1
4,0.946649,0.053351,0
...,...,...,...
93,0.087514,0.912486,1
94,0.152870,0.847130,1
95,0.014701,0.985299,1
96,0.015844,0.984156,1


### 模拟输入

In [10]:
import pickle
from sklearn.model_selection import train_test_split
exclusive = ['Label']
with open('./data/variables-huafen.pkl', 'rb') as file:
    X_resampled, y_resampled = pickle.load(file)
X , y = X_resampled,y_resampled
X

,Subtype I Baseline,Subtype II Baseline,Label
0,0.992366,0.007634,0
1,0.799419,0.200581,0
2,0.039474,0.960526,1
3,0.008023,0.991977,1
4,0.946649,0.053351,0
...,...,...,...
93,0.087514,0.912486,1
94,0.152870,0.847130,1
95,0.014701,0.985299,1
96,0.015844,0.984156,1


In [11]:
_ = 0 
# 步骤1: 分层随机分割数据集
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)
# # 获取划分后的索引
train_indices = X_train.index
test_indices = X_holdout.index
np.savez('split/indices.npz', train=train_indices, test=test_indices)


In [12]:
import numpy as np

data = np.load('./split/indices.npz')

In [13]:
train_PANSS = X_resampled[exclusive].loc[train_indices]
train_PANSS[(train_PANSS != -1).any(axis=1)]

,Label
9,0
8,1
83,1
51,1
93,1
...,...
1,0
58,1
29,1
45,1


In [14]:
test_PANSS = X_resampled[exclusive].loc[test_indices]
print(len(test_PANSS[(test_PANSS != -1).any(axis=1)]))
test_PANSS[(test_PANSS != -1).any(axis=1)]

20


,Label
88,0
46,0
12,0
81,1
43,1
26,0
97,1
21,1
86,1
0,0


In [15]:
test_indices

Int64Index([88, 46, 12, 81, 43, 26, 97, 21, 86, 0, 70, 3, 68, 90, 17, 61, 57,
            71, 28, 7],
           dtype='int64')

In [16]:
print("预处理全部完成！")

预处理全部完成！
